# Login Modules

In [1]:
#Dependencies
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from config import salesusername, salespassword, gmailusername, gmailpassword, salesforcelanding
import urllib.request
from bs4 import BeautifulSoup
print("Dependencies Loaded")
from datetime import date
import pandas as pd

driver = webdriver.Chrome('chromedriver')
print("Driver Initiated")

#Nagivate to Salesforce Login Url
driver.get(salesforcelanding);
userbox = driver.find_element_by_name('j_username')
userbox.send_keys(salesusername)
passbox = driver.find_element_by_name('j_password')
passbox.send_keys(salespassword)
logbutton = driver.find_element_by_name('_eventId_proceed')
logbutton.click()

#Nagivate to Google Suite Login Url
driver.get('https://accounts.google.com/signin/v2');
userbox = driver.find_element_by_class_name('whsOnd.zHQkBf')
userbox.send_keys(gmailusername)
nextbutton = driver.find_element_by_id('identifierNext')
ActionChains(driver).click(nextbutton).perform()
time.sleep(2)
passbox = driver.find_element_by_name('password')
passbox.send_keys(gmailpassword)
nextbutton = driver.find_element_by_id('passwordNext')
ActionChains(driver).click(nextbutton).perform()
time.sleep(2)

#go to gmail home
driver.switch_to.window(driver.window_handles[0])
driver.get('https://mail.google.com/mail/u/0/#inbox')

driver.get(salesforcelanding);
print("Operation Complete")

Dependencies Loaded
Driver Initiated
Operation Complete


# Log Back In (If Salesforce AutoLogs Out)

In [ ]:
driver.switch_to.window(driver.window_handles[0])
driver.get(salesforcelanding);
userbox = driver.find_element_by_name('j_username')
userbox.send_keys(salesusername)
passbox = driver.find_element_by_name('j_password')
passbox.send_keys(salespassword)
logbutton = driver.find_element_by_name('_eventId_proceed')
logbutton.click()

# Recurring Functions

In [ ]:
def emailer(rec_email, rec_subject, rec_body):
    driver.get(f'https://mail.google.com/mail/b/ALGkd0xqd9CZauxZgF5c-S4-P_pJLC1x4Kr3-M9B4dsqS_9gL_s9/u/0/?view=cm&fs=1&tf=1&source=mailto&to={rec_email}')
    time.sleep(3)
    driver.switch_to.window(driver.window_handles[0])
    emailsubject = rec_subject
    emailbody = rec_body
    subjectbox = driver.find_element_by_class_name('aoT')
    subjectbox.send_keys(emailsubject)
    bodybox = driver.find_element_by_id(':nk')
    bodybox.send_keys(emailbody)

def execute_email(emailsubject, emailbody):
    time.sleep(6)
    driver.find_element_by_class_name('aoT').send_keys(emailsubject)
    driver.find_element_by_id(':nk').send_keys(emailbody)
    time.sleep(1)
    driver.find_element_by_class_name('J-J5-Ji.J-JN-M-I-JG').click()
    driver.find_element_by_class_name('J-J5-Ji.J-JN-M-I-JG').click()
    driver.find_element_by_class_name('J-J5-Ji.J-JN-M-I-JG').click()
    time.sleep(2)
    driver.find_element_by_class_name('J-N.J-Ph.CG').click()
    driver.find_element_by_class_name('J-N.J-Ph.CG').click()
    driver.find_element_by_class_name('J-N.J-Ph.CG').click()
    time.sleep(2)
    driver.find_element_by_class_name('J-N.BB').click()
    time.sleep(2)
    driver.find_element_by_id(':oy').click()

def cls_button_locater(html_tables):    
    #find the "Open Activites" table
    for td_tag in range(len(html_tables)):
        if html_tables[td_tag].find('td', class_='pbTitle').get_text() == "Open Activities":
            #find the "Cls" Button
            for t_row in html_tables[td_tag].find_all('tr'):
                for a_tag in t_row.find_all('a'):
                    if a_tag.get_text() == "Cls":
                        driver.get(f"""{salesforcelanding}{str(a_tag).replace('<a class="actionLink" href="/','').split('"')[0]}""")

# Data Gathering Module

In [ ]:
tag = "JC - "
driver.switch_to.window(driver.window_handles[0])
all_fwups = driver.find_elements_by_partial_link_text(f'{tag}')
for fwup in all_fwups:
    ActionChains(driver).key_down(Keys.COMMAND).click(fwup).key_up(Keys.COMMAND).perform()

In [ ]:
dataset = []
for x in driver.window_handles:
    try:
        driver.switch_to.window(driver.window_handles[1])
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        taskdetail = soup.find('h2', class_="mainTitle").get_text()
        if taskdetail == "Task Detail":            
            duedate = soup.find("div", id="tsk4_ileinner").get_text()
            fwup = soup.find("div", id="tsk5_ileinner").get_text()
            driver.find_element_by_xpath("/html/body/div/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/a").click()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
        else:
            duedate = "new lead"
            fwup = ""
        name = soup.find("div", id="con2_ileinner").get_text()
        phone = soup.find("div", id="con12_ileinner").get_text()    
        major = soup.find("div", id="00N3600000LP34V_ileinner").get_text()
        createdby = soup.find('div', id='CreatedBy_ileinner').get_text().split(" ")
        initial_date = createdby[len(createdby) - 3]
        url = driver.current_url
        email = soup.find('div', id='con15_ileinner').get_text()
        #populate dictionary with the mined data
        fwupdict = {
            "fwup":fwup,
            "name":name,
            "major":major,
            "duedate":duedate,
            "initial date":initial_date,
            "phone":phone,
            "email":"",
            "emailfwup":"",
            "calltext":"",
            "comments":"",
            "newtask":"",
            "date":"",
            "time":"",
            "status":"",
            "url":url,
            "leademail":email
        }
        dataset.append(fwupdict)
        driver.close()
    except:
        print('finished')
        break
df = pd.DataFrame(dataset)
df.to_csv(f'fwups{date.today()}{tag}.csv', index=False)

# Autoloop

In [ ]:
newdf = pd.read_csv("fwups2020-11-30newlead.csv")
n = 0
driver.switch_to.window(driver.window_handles[0])
#establish variable to allow for only 3 options; prevent errors
errorprevention = 0
#ask user if they're doing followups or new leads
#New Note Module
x = -1
for url in newdf['url'].tolist():
    driver.get(url)
    x = x + 1
    #make soup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
    #### EEEEEEE  M     M    A   II  LL
    #### EE       MM   MM   A A  II  LL
    #### EEEEEEE  M M M M  AAAAA II  LL
    #### EE       M  M  M  A   A II  LL
    #### EEEEEEE  M     M  A   A II  LLLLLLL
    email_type = newdf['email'].tolist()[x]#input("Press 'Enter' to send email (or 'n' to skip): ")
    email = newdf['leademail'].tolist()[x]
    recipient = soup.find("div", id="con2_ileinner").get_text()
    subjectinput = newdf['major'].tolist()[x]
    subjectoutput = ""
    errorprevention = 0
    while errorprevention < 1:
        if subjectinput == "Data Analytics (M.S.)":
            subjectoutput = "Master of Science in Data Analytics"
            errorprevention = errorprevention + 1
        elif subjectinput == "Data Analytics - Cert":
            subjectoutput = "Certificate in Data Analytics"
            errorprevention = errorprevention + 1
        elif subjectinput == "Cyber Security & Privacy - Cert":
            subjectoutput = "Certificate in Cyber Security"
            errorprevention = errorprevention + 1
        else:
            print('!!!!!!!!!!!!! - Invalid Entry - Retry - !!!!!!!!!!!!!')
            errorprevention = 0
    if email_type == "y":
        driver.get(f'https://mail.google.com/mail/b/ALGkd0xqd9CZauxZgF5c-S4-P_pJLC1x4Kr3-M9B4dsqS_9gL_s9/u/0/?view=cm&fs=1&tf=1&source=mailto&to={email}')
        time.sleep(6)
        #INPUTFIELD
        followup = newdf['emailfwup'].tolist()[x]#input("Followup message? (y/n): ")
        if followup == "y":
            fwupsnippet = f'I just wanted to reach out to you one more time regarding your inquiry into our {subjectoutput}. '
        elif followup == "c":
            fwupsnippet = f"I notice you previously asked for information on the {subjectoutput}. "
        else:
            fwupsnippet = ""
        #INPUTFIELD
        day1 = "Fri"#input("1st day?(___day): ")
        day2 = "Mon"#input("2nd day?(___day): ")
        time1 = "12"#input("1st time?(hh): ")
        time2 = "2"#input("2ndt time?(hh): ")
        time3 = "12"#input("1st time?(hh): ")
        time4 = "4"#input("2ndt time?(hh): ")
#            preventerror_url = driver.current_url
#            while url == driver.current_url:
#                preventerror_url = driver.current_url
#            time.sleep(2)
        if time4 != "":
            driver.switch_to.window(driver.window_handles[0])
            emailsubject = f'Earn a {subjectoutput} at The Catholic University of America'
            emailbody = f"""{recipient}\n\nThank you for your interest in The Catholic University of America!

            My name is John. {fwupsnippet}My role is to offer any help you may need to gather info and start an application.
            To start an application, click here: https://cardinaladmissions.force.com/TX_CommunitiesSelfReg?startURL=%2FTargetX_Portal__PB

            ***Call us for a consultation to redeem your FREE application fee waiver!***
            Please indicate which day in the coming week you would prefer.
            -(1)- {day1}day at {time1}:00pm or {time2}:00pm
            -(2)- {day2}day at {time3}:00pm or {time4}:00pm

            If none of the above times work for you, let me know when you're most available, and I'll schedule you for a special appointment.
            """                
            execute_email(emailsubject, emailbody)
        else:
            print("skipping")
    elif email_type == "s":
        subjectinput = soup.find("div", id="00N3600000LP34V_ileinner").get_text()
        subjectoutput = ""
        driver.get(f'https://mail.google.com/mail/b/ALGkd0xqd9CZauxZgF5c-S4-P_pJLC1x4Kr3-M9B4dsqS_9gL_s9/u/0/?view=cm&fs=1&tf=1&source=mailto&to={email}')
        errorprevention = 0
        while errorprevention < 1:
            if subjectinput == "Data Analytics (M.S.)":
                subjectoutput = "Master of Science in Data Analytics"
                sought_program = "Degree Seeking"
                sought_school = "School of Engineering"
                sought_deg = "Data Analytics (M.S.)"
                errorprevention = errorprevention + 1
            elif subjectinput == "Data Analytics - Cert":
                subjectoutput = "Certificate in Data Analytics"
                sought_program = "Certificate"
                sought_school = "School of Engineering Non-Degree"
                sought_deg = "Data Analytics - Cert"        
                errorprevention = errorprevention + 1
            elif subjectinput == "Cyber Security & Privacy - Cert":
                subjectoutput = "Certificate in Cyber Security"
                sought_program = "Certificate"
                sought_school = "School of Engineering Non-Degree"
                sought_deg = "Cyber Security & Privacy - Cert"        
                errorprevention = errorprevention + 1
            else:
                errorprevention = errorprevention + 1
                subjectoutput = "Master of Science in Data Analytics"
                sought_program = "Degree Seeking"
                sought_school = "School of Engineering"
                sought_deg = "Data Analytics (M.S.)"
                errorprevention = errorprevention + 1
        #Greeting
        driver.get(f'https://mail.google.com/mail/b/ALGkd0xqd9CZauxZgF5c-S4-P_pJLC1x4Kr3-M9B4dsqS_9gL_s9/u/0/?view=cm&fs=1&tf=1&source=mailto&to={email}')
        emailsubject = "Data Analytics at CUA: Info You Requested"
        greet_text = "          My apologies for the delayed delivery of the info you requested.\n\n          I'll be sending you 3 emails in just a moment containing the info you were looking into: specifically - \n\n(1) How to Apply\n(2)Class Overview/Brochure\n(3)Other Offices/Resources at CUA\n\n          Thanks very much for speaking with me. I think you'd make a great candidate and I look forward to hearing from you again soon."
        emailbody = f"{recipient}\n\n{greet_text}"
        execute_email(emailsubject, emailbody)
        #Application Link
        driver.get(f'https://mail.google.com/mail/b/ALGkd0xqd9CZauxZgF5c-S4-P_pJLC1x4Kr3-M9B4dsqS_9gL_s9/u/0/?view=cm&fs=1&tf=1&source=mailto&to={email}')
        emailsubject = f'Applying to Earn a {subjectoutput} at The Catholic University of America'
        app_text = f"          It was a pleasure speaking with you today. Based on our conversation, I think you're a good fit for our Master of Science in Data Analytics Degree program.\n          To begin your application, follow this link: https://cardinaladmissions.force.com/TX_CommunitiesSelfReg?startURL=%2FTargetX_Portal__PB  \n          You'll need to create a login. Once you've logged in, you'll want to make sure to pick the following options from the drop-down menus.\n-{sought_program}\n-{sought_school}\n-{sought_deg}\n-[Desired Semester] (i.e. Spring 2021)\n          If you have any trouble completing the application, please call me at my office# (included below). I can walk you through the process.\n         Thanks very much for speaking with me. I look forward to hearing from you soon!"
        emailbody = f"""{recipient}\n\n{app_text}
        """
        execute_email(emailsubject, emailbody)
        #Brochure
        driver.get(f'https://mail.google.com/mail/b/ALGkd0xqd9CZauxZgF5c-S4-P_pJLC1x4Kr3-M9B4dsqS_9gL_s9/u/0/?view=cm&fs=1&tf=1&source=mailto&to={email}')
        emailsubject = f'Data Analytics at CUA: Email Brochure'
        broch_text = "          To see the presentation we use for our weekly webinar on the program, visit: https://docs.google.com/presentation/d/1CxI03mMTERenu_sI3a6DP3uUH8F4PQkKbdewTSVBTnA/edit?usp=sharing\n\nVisit https://youtu.be/3tKYwsfXIUc to watch a recording of our weekly webinar covering the DA program.\n\nSee a list of available classes below. For more classes and a comprehensive overview, visit: https://metro.catholic.edu/academics/graduate/ms-data-analytics/index.html\n\nCore Classes (First 4/10 classes in the degree)\n-----------------------------------------------------------------\nDA 501: Introduction to Data Science and Python (3 credit hours)\nA hands-on introduction to the field of Data Science and its applications. Covers a wide range of topics to provide an overview of the use of data in different fields. Provides hands-on practice with basic tools and methods of data analysis. Prepares students to use data in their field of study and in their work and to effectively communicate quantitative findings. Focus is on the use of Python in data analysis and mastering tools for acquiring, parsing, manipulating, and preparing data for statistical analysis.\nDA 514: Applied Statistics and Data Analysis (3 credit hours)\nThis is a second course in statistics. The course will focus on Design of experiments and Analysis of Variance, Categorical data analysis, Regression analysis, Time series/forecasting, and Data visualization.\n\nDA 515: Introduction to Machine Learning (3 credit hours)\nThe course introduces principles, algorithms, and applications of machine learning from the point of view of modeling and prediction; formulation of learning problems; representation, over-fitting, generalization; clustering, classification, probabilistic modeling; and methods such as support vector machines, hidden Markov models, and Bayesian networks.\n\nDA 516: Applications of Data Analytics and Development (3 credit hours)\nThe focus will be on programming and data manipulation techniques for constructing analytics-based applications.  Topics include SQL or no-SQL databases, using web service API’s to acquire data, introduction to Hadoop and MapReduce, and use of third-party analytic component API’s.\n-----------------------------------------------------------------\nMSDA Electives (Ask advisors about scheduling for electives in advance; electives are offered on a rotating basis; visit https://metro.catholic.edu/academics/graduate/ms-data-analytics/degree-requirement.html for a full list of electives\n-----------------------------------------------------------------\nCSC 447 - Artificial Intelligence (3 Credit Hours)\nTopics may include state space search, heuristic search, knowledge representation techniques, expert systems, automated reasoning, definitions of intelligence, computer problem solving, game playing, pattern recognition, theorem proving, semantic information, processing, evolutionary systems, and heuristic programming. Prerequisite: Instructor's permission.\n\nCSC 410 - Fundamentals of Cloud Computing (3 Credit Hours)\nOverview of the field of Cloud Computing, its enabling technologies, main building blocks, and hands-on experience through projects utilizing public cloud infrastructures, such as Amazon Web Services (AWS) and Microsoft Azure. Cloud computing services are being adopted widely across a variety of organizations in many domains. Simply, cloud computing is the delivery of computing as a service over a network, whereby distributed resources are rented, rather than owned, by an end user as a utility. Prerequisites: CSC 363 and CSC 323\n\nCSC 728 - Visualization (3 Credit Hours)\nThe course focuses on visualization of scientific data. Both visualization principles and practical design issues are addressed. The course introduces the visualization pipeline. It covers the visualization of scalar data, vector data, and tensor data. It also covers image visualization, volume visualization and finally information visualization. It discusses the effective use of visualization in various areas of the natural sciences, and examples of application will be drawn from these areas. It emphasizes the importance of visualization in understanding observations, examining theories, and fostering new scientific hypothesis. representation and presentation, document visualization, and dynamic exploration Permission of instructor.\n\nCSC 651 - Multimedia Processing and Information Retrieval (3 Credit Hours)\nThis course covers topics including multimedia systems, multimedia applications, image compression and processing, video compression and processing, content-based image retrieval, and content-based video indexing and retrieval. Prerequisite: EE 634 or Permission of instructor.\n\nCSC 641 - Data Mining (3 Credit Hours)\nIntroduction to data mining techniques, including data preprocessing, data mining primitives, association rules, decision trees, cluster analysis, classification and machine learning, data visualization, and data warehousing. Applications from a wide variety of domains will be studied. Prerequisite: CSC 541 or Permission of instructor."
        emailbody = f"{recipient}\n\n{broch_text}"
        execute_email(emailsubject, emailbody)
        #Resources
        driver.get(f'https://mail.google.com/mail/b/ALGkd0xqd9CZauxZgF5c-S4-P_pJLC1x4Kr3-M9B4dsqS_9gL_s9/u/0/?view=cm&fs=1&tf=1&source=mailto&to={email}')
        emailsubject = f'Resources at CUA'
        resource_text = "          Included in this email are some important CUA resources for you to take advantage of.\n\nCAREER SERVICES:\nhttps://success.catholic.edu/about-us/contact-us/index.html\nPhone: 202-319-6262\nEmail: success@cua.edu\nENROLLMENT SERVICES:\nhttps://enrollment-services.catholic.edu/contact-us/index.html\nPhone: 202-319-5300\nEmail: cua-enrollmentservices@cua.edu\n\nFINANCIAL AID SERVICES:\nhttps://financial-aid.catholic.edu/contact-us/index.html\nPhone: 202-319-5307\nEmail: cua-finaid@cua.edu"
        emailbody = f"{recipient}\n\n{resource_text}"
        execute_email(emailsubject, emailbody)
    elif email_type == "d":
        #APPLICATIONS SECTION
        for td_tag in range(len(html_tables)):
            myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
            #find the "Applications" table and save it's contents to the apps_table variable
            if myvar == "Applications":
                apps_table = html_tables[td_tag]
        tdata = apps_table.find_all('tr')[2].find_all('td')
        app_semester = tdata[3].get_text() +" "+ tdata[4].get_text()
        driver.get(f'https://mail.google.com/mail/b/ALGkd0xqd9CZauxZgF5c-S4-P_pJLC1x4Kr3-M9B4dsqS_9gL_s9/u/0/?view=cm&fs=1&tf=1&source=mailto&to={email}')
        emailsubject = f"Your Application to Catholic U's {subjectoutput}"
        deferral_text = f"          This is John, from Catholic University of America. I hope you've been well!\n\n          We worked together on your {app_semester} application for the {subjectoutput} and you had expressed interested in deferring to Spring 2021 and there are still open seats left!\n\n          Please let me know if you'd like to enroll and I may be able to change your old app to Spring 2021. The deadline for submission is December 17th, 2020.\n          Click here to start a new app: https://cardinaladmissions.force.com/\n\n          Thanks so much for your interest in our programs! I hope to speak to you soon."
        emailbody = f"{recipient}\n\n{deferral_text}"
        execute_email(emailsubject, emailbody)
    elif email_type == "n":
        print("Skipping Email")
        emailbody = "(no email sent for this note)"
    else:
        print("Invalid Key. Try Again.")
    print("Emails finished")

    #### NN   NN  OOOOOOO  TTTTTT EEEEEE
    #### NNN  NN  OO   OO    TT   EE
    #### NN N NN  OO   OO    TT   EEEEEE
    #### NN  NNN  OO   OO    TT   EE
    #### NN   NN  OOOOOOO    TT   EEEEEE
    driver.get(url);
    #find subjectline and commentsline strings from the df
    subject = newdf['calltext'].tolist()[x]
    comments = newdf['comments'].to_list()[x]
    #determine the appropriate button to click
    #execute click
    if str(newdf['fwup'].tolist()[x]) != 'nan': 
        cls_button_locater(html_tables)
    else:
        while driver.current_url == url:
            try:
                driver.get(url)
                driver.find_element_by_name('new').click()
                time.sleep(1)
            except:
                print('success')
    #execute note
    driver.find_element_by_name('tsk5').send_keys(f'JC - {subject}')
    driver.find_element_by_name('tsk6').send_keys(f'{comments}\n Email:\n{emailbody}')
    driver.find_element_by_name('save').click()            
    print("Operation Complete")
    driver.get(url)
    #New Task Module
    if str(newdf['newtask'].tolist()[x]) != 'nan':
        preventerror_url = driver.current_url
        #INPUTFIELD
        targetdate = str(newdf['date'].tolist()[x]).replace('"','')#input("What date?(mm/dd): ")
        targettime = str(newdf['time'].tolist()[x])#input('What time?(i.e. "8" or "4"): ')
        subject = newdf['newtask'].tolist()[x]#input("Subject: ")
        driver.switch_to.window(driver.window_handles[n])
        driver.find_element_by_name('task').click()
        while url == driver.current_url:
            preventerror_url = driver.current_url
            try:
                driver.find_element_by_name('task').click()
            except:
                print('still trying')
        print("Task Started")
        driver.find_element_by_name('tsk5').send_keys(f'JC - {subject}')
        print("typed subject")
        driver.find_element_by_name('tsk4').send_keys(f'{targetdate}')
        print("typed date")
        driver.find_element_by_name('reminder_dt_time').send_keys(targettime)
        print("entered time")
        driver.find_elements_by_name('save')[1].click()
        print("Saved")
        print("Operation Complete")
    else:
        print("Skipping New Task")
    #Status Module
    if newdf['status'].tolist()[x] != "nan":
        newstatus = newdf['status'].tolist()[x]
        try:
            driver.switch_to.window(driver.window_handles[0])
            status = driver.find_element_by_id('00N0e000002cYq3_ilecell')
            status.click()
            status.send_keys(Keys.RETURN)
            status.click()
            time.sleep(1)
            driver.find_element_by_id('00N0e000002cYq3').send_keys(newstatus)
            status.click()
            driver.find_element_by_name('inlineEditSave').click()
            time.sleep(5)
            print("Operation Complete")
        except:
            print("Attempt failed; click manually")
    else:
        print('Skipping Status')

# END

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
for td_tag in range(len(html_tables)):
    myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
    #find the "Recommendations" table and save it's contents to the rec_table variable
    if myvar == "Open Activities":
        #tsk_table = html_tables[td_tag]
        for t_row in html_tables[td_tag].find_all('tr'):
            for a_tag in t_row.find_all('a'):
                if a_tag.get_text() == "Cls":
                    print(salesforcelanding+str(a_tag).replace('<a class="actionLink" href="/','').split('"')[0])
a_tags = t_rows[2].find_all('a')
cls_link = str(a_tags[1]).replace('<a class="actionLink" href="','').split('"')[0]
print(f'{salesforcelanding}{cls_link}')

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
html_tables = soup\
              .find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")

for td_tag in range(len(html_tables)):
    myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
    #find the "Recommendations" table and save it's contents to the rec_table variable
    if myvar == "Open Activities":
        #tsk_table = html_tables[td_tag]
        for t_row in html_tables[td_tag].find_all('tr'):
            for a_tag in t_row.find_all('a'):
                if a_tag.get_text() == "Cls":
                    print(salesforcelanding+str(a_tag).replace('<a class="actionLink" href="/','').split('"')[0])
a_tags = t_rows[2].find_all('a')
cls_link = str(a_tags[1]).replace('<a class="actionLink" href="','').split('"')[0]
print(f'{salesforcelanding}{cls_link}')

In [ ]:
for td_tag in range(len(BeautifulSoup(driver.page_source, 'html.parser')\
.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette"))):
    if BeautifulSoup(driver.page_source, 'html.parser')\
    .find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")[td_tag]\
    .find('td', class_='pbTitle').get_text() == "Open Activities":
        for t_row in BeautifulSoup(driver.page_source, 'html.parser')\
        .find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")[td_tag]\
        .find_all('tr'):
            for a_tag in t_row.find_all('a'):
                if a_tag.get_text() == "Cls":
                    print(salesforcelanding+str(a_tag).replace('<a class="actionLink" href="/','').split('"')[0])
a_tags = t_rows[2].find_all('a')
cls_link = str(a_tags[1]).replace('<a class="actionLink" href="','').split('"')[0]
print(f'{salesforcelanding}{cls_link}')

In [ ]:
listylist = "0 1 2"\
            .split(' ')
for litem in\
    listylist:
    print(litem)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
for td_tag in range(len(html_tables)):
    myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
    #find the "Activity History" table and save it's contents to the rec_table variable
    if myvar == "Activity History":
        for t_row in html_tables[td_tag].find_all('tr'):
            for in_tag in t_row.find_all('input'):
                if str(in_tag).replace('<input class="btn" ','').split('" ')[0] == 'name="new':
                    driver.get(f'''{salesforcelanding}{str(in_tag).replace("""<input class="btn" name="new" onclick="navigateToUrl('/""",'').split('" title=')[0]}''')
                    

#            
#                if in_tag.get_text() == "Cls":
#                    print(salesforcelanding+str(a_tag).replace('<a class="actionLink" href="/','').split('"')[0])

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
for td_tag in range(len(html_tables)):
    myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
    #find the "Recommendations" table and save it's contents to the rec_table variable
    if myvar == "Open Activities":
        tsk_table = html_tables[td_tag]
t_rows = tsk_table.find_all('tr')
a_tags = t_rows[2].find_all('a')
str(a_tags[1]).replace('<a class="actionLink" href="','').split('"')[0]

In [ ]:
subject = newdf['calltext'].tolist()[x]
comments = newdf['comments'].to_list()[x]

if close_or_new =="c":
    logbutton = "Cls"
elif close_or_new =="l":
    logbutton = "Log a Call"

preventerror_url = driver.current_url
driver.find_element_by_partial_link_text(logbutton).click()
while url == driver.current_url:
    preventerror_url = driver.current_url
    try:
        driver.find_element_by_partial_link_text(logbutton).click()
    except:
        print('still trying')
driver.find_element_by_name('tsk5').send_keys(f'JC - {subject}')
driver.find_element_by_name('tsk6').send_keys(f'{comments}\n Email:\n{emailbody}')
driver.find_element_by_name('save').click()

In [ ]:

soup = BeautifulSoup(driver.page_source, 'html.parser')
nextbutton = soup.find('div', id='sbar-l-wrp').find('div', class_='b-pager').find('a', class_='b-pager-next').get_text()
text = ""
while nextbutton == 'Next':
    try:
        text = str(soup.find('div', id="content").find('div', class_="b-story-body-x x-r15").find("p")).replace("<p>","").replace("</p>","").replace("<em>","").replace("</em>","").replace("<br>","\n").replace("<br/>","")
        text = text + storytext
        driver.find_element_by_class_name('b-pager-next').click()
        nextbutton = soup.find('div', id='sbar-l-wrp').find('div', class_='b-pager').find('a', class_='b-pager-next').get_text()
    except:
        print('an exception occurred')
print(text)

In [ ]:
driver.switch_to.window(driver.window_handles[1])
soup = BeautifulSoup(driver.page_source, 'html.parser')
rec_email = soup.find('div', id='ToAddress_ileinner').get_text()
rec_subject = soup.find('div', id='Subject_ileinner').get_text()
rec_body = soup.find('div', id='TextBody_ileinner').get_text()
print(rec_email)
print(rec_subject)
print(rec_body)

In [ ]:
driver.switch_to.window(driver.window_handles[0])
soup = BeautifulSoup(driver.page_source, 'html.parser')
#find all of the tables in the html and put them in a list
html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
#search the list for the headers/titles of each table;
for td_tag in range(len(html_tables)):
    myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
    #find the "Recommendations" table and save it's contents to the rec_table variable
    if myvar == "Recommendations":
        rec_table = html_tables[td_tag]
#double check that rec_table is the right one
table_name = rec_table.find('td', class_='pbTitle').get_text()
#find all table rows inside of the recs table and save them to a list.
t_rows = rec_table.find_all('tr')
#t_rows = t_rows.pop(0)
#print(t_rows)

In [ ]:
ctr = 0
incomp_rows = []
for t_row in t_rows:
#    print(f'\n\nindex: {ctr} {t_row}')
    t_data = t_row.find_all('td', class_="dataCell")
    for t_datum in t_data:
#        print(t_datum)
        if t_datum.get_text() == "Incomplete":
#            print(f"incomplete found at: {ctr} {t_datum.get_text()}")
            incomp_rows.append(t_row)
    ctr = ctr + 1

In [ ]:
open_recs = []
for t_row in incomp_rows:
    t_hrefs = t_row.find_all('a')
    href = str(t_hrefs).split('">Recommendation</a>,').pop(0).replace('[<a href="/','')
    open_recs.append(href)
open_recs
#f'{salesforcelanding}{href}'

In [ ]:
#driver.switch_to.window(driver.window_handles[0])
soup = BeautifulSoup(driver.page_source, 'html.parser')
#find all of the tables in the html and put them in a list
html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
#search the list for the headers/titles of each table;
for td_tag in range(len(html_tables)):
    myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
    #find the "Recommendations" table and save it's contents to the rec_table variable
    if myvar == "Recommendations":
        rec_table = html_tables[td_tag]
#double check that rec_table is the right one
table_name = rec_table.find('td', class_='pbTitle').get_text()
#find all table rows inside of the recs table and save them to a list.
t_rows = rec_table.find_all('tr')
#t_rows = t_rows.pop(0)
#print(t_rows)
incomp_rows = []
for t_row in t_rows:
    t_data = t_row.find_all('td', class_="dataCell")
    for t_datum in t_data:
#        print(t_datum)
        if t_datum.get_text() == "Incomplete":
            incomp_rows.append(t_row)
open_recs = []
for t_row in incomp_rows:
    t_hrefs = t_row.find_all('a')
    href = str(t_hrefs).split('">Recommendation</a>,').pop(0).replace('[<a href="/','')
    open_recs.append(href)
open_recs
#f'{salesforcelanding}{href}'

In [ ]:
def emailer(rec_email, rec_subject, rec_body):
    driver.get(f'https://mail.google.com/mail/b/ALGkd0xqd9CZauxZgF5c-S4-P_pJLC1x4Kr3-M9B4dsqS_9gL_s9/u/0/?view=cm&fs=1&tf=1&source=mailto&to={rec_email}')
    time.sleep(3)
    driver.switch_to.window(driver.window_handles[0])
    emailsubject = rec_subject
    emailbody = rec_body
    subjectbox = driver.find_element_by_class_name('aoT')
    subjectbox.send_keys(emailsubject)
    bodybox = driver.find_element_by_id(':nk')
    bodybox.send_keys(emailbody)
    time.sleep(1)
    driver.find_element_by_class_name('J-J5-Ji.J-JN-M-I-JG').click()
    driver.find_element_by_class_name('J-J5-Ji.J-JN-M-I-JG').click()
    driver.find_element_by_class_name('J-J5-Ji.J-JN-M-I-JG').click()
    time.sleep(2)
    driver.find_element_by_class_name('J-N.J-Ph.CG').click()
    driver.find_element_by_class_name('J-N.J-Ph.CG').click()
    driver.find_element_by_class_name('J-N.J-Ph.CG').click()
    time.sleep(2)
    driver.find_element_by_class_name('J-N.BB').click()
    time.sleep(2)
    driver.find_element_by_id(':oy').click()
    time.sleep(5)

for rec in open_recs:
    driver.get(f'{salesforcelanding}{rec}')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    grab_email = str(soup).split('<a href="mailto:')
    rec_email = grab_email[1].split('"')[0]
    #driver.switch_to.window(driver.window_handles[0])
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #find all of the tables in the html and put them in a list
    html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
    #search the list for the headers/titles of each table;
    for td_tag in range(len(html_tables)):
        myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
        #find the "Recommendations" table and save it's contents to the rec_table variable
        if myvar == "Activity History":
            rec_table = html_tables[td_tag]
    #double check that rec_table is the right one
    table_name = rec_table.find('td', class_='pbTitle').get_text()
    #find all table rows inside of the recs table and save them to a list.
    t_body = rec_table.find_all('tbody')[1]
    #print(t_body.prettify())
    rec_href = str(t_body.find('a')).split('"')[1]
    driver.switch_to.window(driver.window_handles[0])
    driver.find_element_by_name('new').click()
    while url == driver.current_url:
        preventerror_url = driver.current_url
    driver.find_element_by_name('tsk5').send_keys(f'JC - Email: {rec_subject}')
    driver.find_element_by_name('save').click()
    driver.get(f'{salesforcelanding}{rec_href}')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    rec_email = soup.find('div', id='ToAddress_ileinner').get_text()
    rec_subject = soup.find('div', id='Subject_ileinner').get_text()
    rec_body = soup.find('div', id='TextBody_ileinner').get_text()
    emailer(rec_email, rec_subject, rec_body)

In [ ]:
driver.switch_to.window(driver.window_handles[0])
driver.find_element_by_name('new').click()
while url == driver.current_url:
    preventerror_url = driver.current_url
driver.find_element_by_name('tsk5').send_keys(f'JC - Email: {rec_subject}')
driver.find_element_by_name('save').click()

In [ ]:
#driver.switch_to.window(driver.window_handles[0])
soup = BeautifulSoup(driver.page_source, 'html.parser')
#find all of the tables in the html and put them in a list
html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
#search the list for the headers/titles of each table;
for td_tag in range(len(html_tables)):
    myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
    #find the "Recommendations" table and save it's contents to the rec_table variable
    if myvar == "Activity History":
        rec_table = html_tables[td_tag]
#double check that rec_table is the right one
table_name = rec_table.find('td', class_='pbTitle').get_text()
#find all table rows inside of the recs table and save them to a list.
t_body = rec_table.find_all('tbody')[1]
#print(t_body.prettify())
rec_href = str(t_body.find('a')).split('"')[1]

In [ ]:
#driver.switch_to.window(driver.window_handles[0])
soup = BeautifulSoup(driver.page_source, 'html.parser')
#find all of the tables in the html and put them in a list
html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
#search the list for the headers/titles of each table;
for td_tag in range(len(html_tables)):
    myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
    #find the "Recommendations" table and save it's contents to the rec_table variable
    if myvar == "Recommendations":
        rec_table = html_tables[td_tag]
#double check that rec_table is the right one
table_name = rec_table.find('td', class_='pbTitle').get_text()
#find all table rows inside of the recs table and save them to a list.
t_rows = rec_table.find_all('tr')
#t_rows = t_rows.pop(0)
#print(t_rows)
incomp_rows = []
for t_row in t_rows:
    t_data = t_row.find_all('td', class_="dataCell")
    for t_datum in t_data:
#        print(t_datum)
        if t_datum.get_text() == "Incomplete":
            incomp_rows.append(t_row)
open_recs = []
for t_row in incomp_rows:
    t_hrefs = t_row.find_all('a')
    href = str(t_hrefs).split('">Recommendation</a>,').pop(0).replace('[<a href="/','')
    open_recs.append(href)
open_recs
#f'{salesforcelanding}{href}'

preventerror_url = driver.current_url
driver.find_element_by_name('new').click()
while url == driver.current_url:
    preventerror_url = driver.current_url
    driver.find_element_by_name('new').click()
time.sleep(2)
driver.find_element_by_name('tsk5').send_keys(f'JC - sent request emails for rec letters')
driver.find_element_by_name('save').click()


def emailer(rec_email, rec_subject, rec_body):
    driver.get(f'https://mail.google.com/mail/b/ALGkd0xqd9CZauxZgF5c-S4-P_pJLC1x4Kr3-M9B4dsqS_9gL_s9/u/0/?view=cm&fs=1&tf=1&source=mailto&to={rec_email}')
    time.sleep(3)
    driver.switch_to.window(driver.window_handles[0])
    emailsubject = rec_subject
    emailbody = rec_body
    subjectbox = driver.find_element_by_class_name('aoT')
    subjectbox.send_keys(emailsubject)
    bodybox = driver.find_element_by_id(':nk')
    bodybox.send_keys(emailbody)
    time.sleep(1)
    driver.find_element_by_class_name('J-J5-Ji.J-JN-M-I-JG').click()
    driver.find_element_by_class_name('J-J5-Ji.J-JN-M-I-JG').click()
    driver.find_element_by_class_name('J-J5-Ji.J-JN-M-I-JG').click()
    time.sleep(2)
    driver.find_element_by_class_name('J-N.J-Ph.CG').click()
    driver.find_element_by_class_name('J-N.J-Ph.CG').click()
    driver.find_element_by_class_name('J-N.J-Ph.CG').click()
    time.sleep(2)
    driver.find_element_by_class_name('J-N.BB').click()
    time.sleep(2)
#    driver.find_element_by_id(':oy').click()
    time.sleep(5)

for rec in open_recs:
    driver.get(f'{salesforcelanding}{rec}')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    grab_email = str(soup).split('<a href="mailto:')
    rec_email = grab_email[1].split('"')[0]
    #driver.switch_to.window(driver.window_handles[0])
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #find all of the tables in the html and put them in a list
    html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
    #search the list for the headers/titles of each table;
    for td_tag in range(len(html_tables)):
        myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
        #find the "Recommendations" table and save it's contents to the rec_table variable
        if myvar == "Activity History":
            rec_table = html_tables[td_tag]
    #double check that rec_table is the right one
    table_name = rec_table.find('td', class_='pbTitle').get_text()
    #find all table rows inside of the recs table and save them to a list.
    t_body = rec_table.find_all('tbody')[1]
    #print(t_body.prettify())
    rec_href = str(t_body.find('a')).split('"')[1]
    preventerror_url = driver.current_url
    time.sleep(2)
    driver.find_element_by_name('new').click()
    while url == driver.current_url:
        preventerror_url = driver.current_url
    driver.find_element_by_name('tsk5').send_keys(f'JC - Email: Recommendation Requested')
    driver.find_element_by_name('save').click()
    driver.get(f'{salesforcelanding}{rec_href}')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    rec_email = soup.find('div', id='ToAddress_ileinner').get_text()
    rec_subject = soup.find('div', id='Subject_ileinner').get_text()
    rec_body = soup.find('div', id='TextBody_ileinner').get_text()
    emailer(rec_email, rec_subject, rec_body)

In [ ]:
from datetime import datetime
preventerror_url = driver.current_url
driver.find_element_by_name('new').click()
print(f'{datetime}')
while url == driver.current_url:
    print(f'{datetime}')
    preventerror_url = driver.current_url
    print(f'{datetime}')
driver.find_element_by_name('tsk5').send_keys(f'JC - sent request emails for rec letters')
print(f'{datetime}')
driver.find_element_by_name('save').click()
print(f'{datetime}')

In [ ]:
#driver.get('https://cardinaladmissions-manager.my.salesforce.com/003?fcf=00B0e0000089y2h')
#time.sleep(5)
soup = BeautifulSoup(driver.page_source, 'html.parser')
name_hrefs = soup.find_all('div', class_="x-grid3-cell-inner x-grid3-col-FULL_NAME")
len(name_hrefs)

In [ ]:
while len(name_hrefs) > 0:
    href = str(name_hrefs[0]).split('><a href="/')[1].split('"')[0]
    print(f'{x}:{href}')
    driver.get(f'{salesforcelanding}{href}')
    #find all of the tables in the html and put them in a list
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
    for td_tag in range(len(html_tables)):
        myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
        #find the "Applications" table and save it's contents to the apps_table variable
        if myvar == "Applications":
            apps_table = html_tables[td_tag]
    #double check that apps_table is the right one
    table_name = apps_table.find('td', class_='pbTitle').get_text()
    print(table_name)
    apps = apps_table.find_all('tr')#.find_all('th')
#    print(apps.prettify())    
#    for app in apps:
#        print(app.prettify())
    for x in range(len(apps)-2):
        app = apps[x+2]
        app_cols = app.find_all('td')
        appdata = f'{app_cols[0].get_text()}    {app_cols[1].get_text()}    {app_cols[2].get_text()}    {app_cols[3].get_text()}    {app_cols[4].get_text()}'
        print(appdata)
    print('----------------------------------------------')
    #search the list for the headers/titles of each table;
    for td_tag in range(len(html_tables)):
        myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
        #find the "Activity History" table and save it's contents to the acts_table variable
        if myvar == "Activity History":
            acts_table = html_tables[td_tag]
    #double check that acts_table is the right one
    table_name = acts_table.find('td', class_='pbTitle').get_text()
    print(table_name)
    notes = acts_table.find_all('tr')#.find_all('th')
    for x in range(len(notes)-2):
        note = notes[x+2]
        note_text = note.find('th', class_='dataCell').get_text()
        note_date = note.find('td', class_='dataCell DateElement').get_text()
        print('----------------------------------------------')
        print(f'{note_date}: {note_text}')
    print('----------------------------------------------')
    #Status Module
    #INPUTFIELD
    newstatus = input("Enter Appropriate Status: ")
#        driver.switch_to.window(driver.window_handles[0])
    status = driver.find_element_by_id('00N0e000002cYq3_ilecell')
    status.click()
    status.send_keys(Keys.RETURN)
    try:
        status.click()
        time.sleep(1)
        driver.find_element_by_id('00N0e000002cYq3').send_keys(newstatus)
        status.click()
        driver.find_element_by_name('inlineEditSave').click()
        time.sleep(2)
        print("Operation Complete")
    except:
        try:
            status.click()
            driver.find_element_by_name('inlineEditSave').click()
            time.sleep(2)
        except:
            try:
                status.click()
                driver.find_element_by_name('inlineEditSave').click()
                time.sleep(2)
            except:
                print("Attempt failed; click manually")
    reviewed_leads.append(href)
    name_hrefs.pop(0)

In [ ]:
preserveident = 0
if preserveident == 0:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
    #search the list for the headers/titles of each table;
    for td_tag in range(len(html_tables)):
        myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
        #find the "Activity History" table and save it's contents to the acts_table variable
        if myvar == "Activity History":
            acts_table = html_tables[td_tag]
    #double check that acts_table is the right one
    table_name = acts_table.find('td', class_='pbTitle').get_text()
    print(table_name)
    notes = acts_table.find_all('tr')#.find_all('th')
    for x in range(len(notes)-2):
        note = notes[x+2]
        note_text = note.find('th', class_='dataCell').get_text()
        note_date = note.find('td', class_='dataCell DateElement').get_text()
        print('----------------------------------------------')
        print(f'{note_date}: {note_text}')
    print('----------------------------------------------')

    

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
preserveident = 0
if preserveident == 0:
    html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
    #search the list for the headers/titles of each table;
    for td_tag in range(len(html_tables)):
        myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
        #find the "Applications" table and save it's contents to the apps_table variable
        if myvar == "Applications":
            apps_table = html_tables[td_tag]
    #double check that apps_table is the right one
    table_name = apps_table.find('td', class_='pbTitle').get_text()
    print(table_name)
    apps = apps_table.find_all('tr')#.find_all('th')
#    print(apps.prettify())    
#    for app in apps:
#        print(app.prettify())
    for x in range(len(apps)-2):
        app = apps[x+2]
        app_cols = app.find_all('td')
        print(f'{app_cols[0].get_text()}    {app_cols[1].get_text()}    {app_cols[2].get_text()}    {app_cols[3].get_text()}    {app_cols[4].get_text()}')


In [ ]:
mylist = [1,2,3,4,5,6,7,8,9,10]
while len(mylist) > 0:
    print(mylist[0])
    mylist.pop(0)

In [ ]:
statlist = []

In [ ]:
len(statlistarchive)

In [ ]:
#driver.get('https://cardinaladmissions-manager.my.salesforce.com/003?fcf=00B0e0000089y2h')
#time.sleep(5)
soup = BeautifulSoup(driver.page_source, 'html.parser')
name_hrefs = soup.find_all('div', class_="x-grid3-cell-inner x-grid3-col-FULL_NAME")
len(name_hrefs)

In [ ]:
#RESUME HERE
driver.switch_to.window(driver.window_handles[0])
href = str(name_hrefs[0]).split('><a href="/')[1].split('"')[0]
driver.get(f'{salesforcelanding}{href}')
while len(name_hrefs) > 0:
#    #NAVIGATION MODULE
    href = str(name_hrefs[0]).split('><a href="/')[1].split('"')[0]
    print(f'currentpage:{href}')
    driver.get(f'{salesforcelanding}{href}')

    #find all of the tables in the html and put them in a list
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    html_tables = soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")
    #APPLICATIONS SECTION
    for td_tag in range(len(html_tables)):
        myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
        #find the "Applications" table and save it's contents to the apps_table variable
        if myvar == "Applications":
            apps_table = html_tables[td_tag]
    #double check that apps_table is the right one
    table_name = apps_table.find('td', class_='pbTitle').get_text()
    print(table_name)
    apps = apps_table.find_all('tr')#.find_all('th')
#    print(apps.prettify())    
#    for app in apps:
#        print(app.prettify())
    for x in range(len(apps)-2):
        app = apps[x+2]
        app_cols = app.find_all('td')
        print(f'{app_cols[0].get_text()}    {app_cols[1].get_text()}    {app_cols[2].get_text()}    {app_cols[3].get_text()}    {app_cols[4].get_text()}')
    print('----------------------------------------------')
    
    #OPEN ACTIVITIES SECTION
    for td_tag in range(len(html_tables)):
        myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
        #find the "Activity History" table and save it's contents to the acts_table variable
        if myvar == "Open Activities":
            acts_table = html_tables[td_tag]
    #double check that acts_table is the right one
    table_name = acts_table.find('td', class_='pbTitle').get_text()
    print(table_name)
    notes = acts_table.find_all('tr')#.find_all('th')
    for x in range(len(notes)-2):
        note = notes[x+2]
        note_text = note.find('th', class_='dataCell').get_text()
        note_date = note.find('td', class_='dataCell DateElement').get_text()
        print('----------------------------------------------')
        print(f'{note_date}: {note_text}')
    print('----------------------------------------------')

    #search the list for the headers/titles of each table;
    for td_tag in range(len(html_tables)):
        myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
        #find the "Activity History" table and save it's contents to the acts_table variable
        if myvar == "Activity History":
            acts_table = html_tables[td_tag]
    #double check that acts_table is the right one
    table_name = acts_table.find('td', class_='pbTitle').get_text()
    print(table_name)
    notes = acts_table.find_all('tr')#.find_all('th')
    for x in range(len(notes)-2):
        note = notes[x+2]
        note_text = note.find('th', class_='dataCell').get_text()
        note_date = note.find('td', class_='dataCell DateElement').get_text()
        print('----------------------------------------------')
        print(f'{note_date}: {note_text}')
    print('----------------------------------------------')
    print(len(name_hrefs))
    #Status Module
    #INPUTFIELD
    oldstatus = soup.find("div", id="00N0e000002cYq3_ileinner").get_text()
    if oldstatus != "Contacted":
        newstatus = oldstatus
    else:
        newstatus = input("Enter Appropriate Status: ")
    statdict = {"href":href,
                "newstatus":newstatus
               }
    statlist.append(statdict)
    name_hrefs.pop(0)

In [ ]:
print(statlist[len(statlist)-2])
print(statlist[len(statlist)-1])

In [ ]:
len(statlist)-40

In [ ]:
newnewstatlist

In [ ]:
thissoup = BeautifulSoup(driver.page_source, 'html.parser')
if thissoup.find("div", id="00N0e000002cYq3_ileinner").get_text() != "Contacted":
    print("True")
else:
    print("False")


In [ ]:
for stat in statlist:
    href = stat["href"]
    newstatus = stat["newstatus"]
    print(href+"  "+newstatus)

In [ ]:
driver.switch_to.window(driver.window_handles[0])
for stat in newnewstatlist:
    href = stat["href"]
    newstatus = stat["newstatus"]
    driver.get(f'{salesforcelanding}{href}')
    status = driver.find_element_by_id('00N0e000002cYq3_ilecell')
    status.click()
    status.send_keys(Keys.RETURN)
    try:
        status.click()
        time.sleep(1)
        driver.find_element_by_id('00N0e000002cYq3').send_keys(newstatus)
        status.click()
        driver.find_element_by_name('inlineEditSave').click()
        time.sleep(2)
        print("Operation Complete")
    except:
        try:
            status.click()
            driver.find_element_by_name('inlineEditSave').click()
            time.sleep(2)
        except:
            try:
                status.click()
                driver.find_element_by_name('inlineEditSave').click()
                time.sleep(2)
            except:
                print("Attempt failed; click manually")

In [ ]:
appdata = f'{app_cols[0].get_text()}    {app_cols[1].get_text()}    {app_cols[2].get_text()}    {app_cols[3].get_text()}    {app_cols[4].get_text()}'
print(appdata)

In [ ]:
datelist = []
for x in df['Created Date'].to_list():
    h = x.split('-')
    datelist.append(h[0]+"-"+h[1])
datedf = pd.DataFrame({"Created Date":datelist,
                      "Application Number":df["Application Number"]})
datedf

# Appendix 1: Set Up Google Calendar Appt (Mass Invite)

In [ ]:
#Google has no way to send mass invites to calendar appointments.
#This module allows me to do that. Not critical to workflow, but helpful

#Literally copy and paste a single column of email addresses from a csv/xls file
names = """
""".replace('\n','\n\n\n')
#Navigate manually to the desired calendar page before executing this code
driver.switch_to.window(driver.window_handles[0])
#time.sleep for 5 secs to allow operator to prepare
time.sleep(5)
#locate a valid field
box = driver.find_element_by_class_name("T2Ybvb.KRoqRc.editable")
#manually click the invite field; this is the only way to enter data into it using Python;
#the field refuses to respond to selenium attempts at targetting it directly
f = "\n"
box.send_keys(f'3{f*5}2{f*5}1{f*20}{names}')

# Close Out App

In [ ]:
driver.quit()